In [1]:
# Test cell - Run this first
import pandas as pd
import numpy as np
import sklearn
import tensorflow

print("✅ All packages work!")
print(f"Pandas: {pd.__version__}")
print(f"NumPy: {np.__version__}")
print(f"Scikit-learn: {sklearn.__version__}")
print(f"TensorFlow: {tensorflow.__version__}")

✅ All packages work!
Pandas: 2.2.2
NumPy: 2.0.2
Scikit-learn: 1.6.1
TensorFlow: 2.19.0


In [2]:
# ========================================
# LOAD CSV FROM GOOGLE DRIVE
# ========================================

from google.colab import drive
import pandas as pd

# Mount Google Drive
print("📁 Mounting Google Drive...")
drive.mount('/content/drive')
print("✅ Drive mounted!\n")

# ============================================
# CONFIGURE YOUR FILE PATH HERE
# ============================================
FILE_PATH = '/content/drive/MyDrive/gallstone_data.csv'  # ⬅️ CHANGE THIS TO YOUR FILE PATH

# ============================================

# Load the CSV file
print(f"📂 Loading file from: {FILE_PATH}")
print("⏳ Reading CSV file...\n")

df = pd.read_csv(FILE_PATH)

# Clean the data
df = df.dropna(how='all')  # Remove completely empty rows
df = df.dropna(axis=1, how='all')  # Remove completely empty columns
df.columns = [str(col).strip() for col in df.columns]  # Clean column names

print("="*60)
print("🎉 SUCCESS! DATA LOADED FROM DRIVE!")
print("="*60)

print(f"Shape: {df.shape[0]} rows × {df.shape[1]} columns")
print(f"\n📋 Columns:")
print(df.columns.tolist())
print(f"\n📊 First 5 rows:")
print(df.head())
print(f"\n📊 Data types:")
print(df.dtypes)

print("✅ Done!")

📁 Mounting Google Drive...
Mounted at /content/drive
✅ Drive mounted!

📂 Loading file from: /content/drive/MyDrive/gallstone_data.csv
⏳ Reading CSV file...

🎉 SUCCESS! DATA LOADED FROM DRIVE!
Shape: 319 rows × 39 columns

📋 Columns:
['Gallstone Status', 'Age', 'Gender', 'Comorbidity', 'Coronary Artery Disease (CAD)', 'Hypothyroidism', 'Hyperlipidemia', 'Diabetes Mellitus (DM)', 'Height', 'Weight', 'Body Mass Index (BMI)', 'Total Body Water (TBW)', 'Extracellular Water (ECW)', 'Intracellular Water (ICW)', 'Extracellular Fluid/Total Body Water (ECF/TBW)', 'Total Body Fat Ratio (TBFR) (%)', 'Lean Mass (LM) (%)', 'Body Protein Content (Protein) (%)', 'Visceral Fat Rating (VFR)', 'Bone Mass (BM)', 'Muscle Mass (MM)', 'Obesity (%)', 'Total Fat Content (TFC)', 'Visceral Fat Area (VFA)', 'Visceral Muscle Area (VMA) (Kg)', 'Hepatic Fat Accumulation (HFA)', 'Glucose', 'Total Cholesterol (TC)', 'Low Density Lipoprotein (LDL)', 'High Density Lipoprotein (HDL)', 'Triglyceride', 'Aspartat Aminotrans

In [3]:
# ========================================
# GALLSTONE ML ANALYSIS - COMPLETE PIPELINE
# ========================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (confusion_matrix, accuracy_score, precision_score,
                             recall_score, f1_score, roc_curve, roc_auc_score)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout
import warnings
warnings.filterwarnings('ignore')

print("="*60)
print("GALLSTONE PREDICTION - MACHINE LEARNING ANALYSIS")
print("="*60)

GALLSTONE PREDICTION - MACHINE LEARNING ANALYSIS


In [4]:
print(f"\nDataset Shape: {df.shape}")
print(f"Target Variable: Gallstone Status")
print(f"  - 0: Gallstone Present")
print(f"  - 1: No Gallstone")


Dataset Shape: (319, 39)
Target Variable: Gallstone Status
  - 0: Gallstone Present
  - 1: No Gallstone


In [7]:
# ========================================
# DATA PREPROCESSING
# ========================================
print("\n🔧 Preprocessing data...")

# Separate features and target
X = df.drop('Gallstone Status', axis=1)
y = df['Gallstone Status']

print(f"  - Features: {X.shape[1]} columns")
print(f"  - Samples: {X.shape[0]} rows")
print(f"  - Class distribution:")
print(y.value_counts())



🔧 Preprocessing data...
  - Features: 38 columns
  - Samples: 319 rows
  - Class distribution:
Gallstone Status
0    161
1    158
Name: count, dtype: int64


In [25]:
# First split: Train+Val vs Test (75% vs 25%)
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

# Second split: Train vs Val (80% vs 20% of trainval)
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.2, random_state=42, stratify=y_trainval
)

print(f"  - Training: {X_train.shape[0]} samples")
print(f"  - Validation: {X_val.shape[0]} samples")
print(f"  - Testing: {X_test.shape[0]} samples")

  - Training: 191 samples
  - Validation: 48 samples
  - Testing: 80 samples


In [9]:
# ========================================
# NORMALIZATION
# ========================================
print("\n📊 Applying Normalization...")

# Z-Score
scaler_zscore = StandardScaler()
X_train_zscore = scaler_zscore.fit_transform(X_train)
X_test_zscore = scaler_zscore.transform(X_test)

# Min-Max
scaler_minmax = MinMaxScaler()
X_train_minmax = scaler_minmax.fit_transform(X_train)
X_test_minmax = scaler_minmax.transform(X_test)

print("  ✓ Z-Score normalization applied")
print("  ✓ Min-Max normalization applied")


📊 Applying Normalization...
  ✓ Z-Score normalization applied
  ✓ Min-Max normalization applied


In [10]:
# ========================================
# INITIALIZE STORAGE
# ========================================
results = {
    'Model': [], 'Normalization': [], 'Accuracy': [],
    'Precision': [], 'Recall': [], 'F1-Score': [], 'ROC-AUC': []
}
confusion_matrices = {}
roc_curves = {}

In [11]:
# ========================================
# TRAIN MODELS - MIN-MAX
# ========================================
print("\n" + "="*60)
print("🟢 MIN-MAX NORMALIZATION MODELS")
print("="*60)

In [12]:
# 1D CNN
print("\n4️⃣  1D CNN...")
X_train_cnn = X_train_minmax.reshape(X_train_minmax.shape[0], X_train_minmax.shape[1], 1)
X_test_cnn = X_test_minmax.reshape(X_test_minmax.shape[0], X_test_minmax.shape[1], 1)

cnn = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(X_train_minmax.shape[1], 1)),
    Conv1D(32, 3, activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
cnn.fit(X_train_cnn, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=0)

y_proba = cnn.predict(X_test_cnn, verbose=0).ravel()
y_pred = (y_proba > 0.5).astype(int)

results['Model'].append('1D CNN')
results['Normalization'].append('Min-Max')
results['Accuracy'].append(accuracy_score(y_test, y_pred))
results['Precision'].append(precision_score(y_test, y_pred))
results['Recall'].append(recall_score(y_test, y_pred))
results['F1-Score'].append(f1_score(y_test, y_pred))
results['ROC-AUC'].append(roc_auc_score(y_test, y_proba))
confusion_matrices['1D CNN (Min-Max)'] = confusion_matrix(y_test, y_pred)
fpr, tpr, _ = roc_curve(y_test, y_proba)
roc_curves['1D CNN'] = (fpr, tpr, results['ROC-AUC'][-1])
print(f"   Accuracy: {results['Accuracy'][-1]:.4f}")


4️⃣  1D CNN...
   Accuracy: 0.8250


In [26]:
# ========================================
# CORRECT CNN HYPERPARAMETER TUNING
# ========================================

import tensorflow as tf
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [27]:
# ========================================
# STEP 1: PROPER DATA SPLITTING
# ========================================
print("📊 Splitting data properly...")

# First split: Train+Val vs Test (75% vs 25%)
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

# Second split: Train vs Val (80% vs 20% of trainval)
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.2, random_state=42, stratify=y_trainval
)

print(f"  - Training: {X_train.shape[0]} samples")
print(f"  - Validation: {X_val.shape[0]} samples")
print(f"  - Testing: {X_test.shape[0]} samples (NEVER TOUCHED UNTIL FINAL EVALUATION)")

📊 Splitting data properly...
  - Training: 191 samples
  - Validation: 48 samples
  - Testing: 80 samples (NEVER TOUCHED UNTIL FINAL EVALUATION)


In [28]:
# ========================================
# STEP 2: NORMALIZE DATA
# ========================================
print("\n🔧 Normalizing data...")

scaler_minmax = MinMaxScaler()
X_train_norm = scaler_minmax.fit_transform(X_train)
X_val_norm = scaler_minmax.transform(X_val)
X_test_norm = scaler_minmax.transform(X_test)

# Reshape for CNN
X_train_cnn = X_train_norm.reshape(X_train_norm.shape[0], X_train_norm.shape[1], 1)
X_val_cnn = X_val_norm.reshape(X_val_norm.shape[0], X_val_norm.shape[1], 1)
X_test_cnn = X_test_norm.reshape(X_test_norm.shape[0], X_test_norm.shape[1], 1)

print("  ✓ Data normalized and reshaped")


🔧 Normalizing data...
  ✓ Data normalized and reshaped


In [29]:
# ========================================
# STEP 3: BUILD MODEL FUNCTION
# ========================================
def build_model(hp):
    model = Sequential()

    # First Conv layer
    model.add(Conv1D(
        filters=hp.Int('conv1_filters', min_value=32, max_value=128, step=32),
        kernel_size=hp.Choice('conv1_kernel', values=[2, 3, 5]),
        activation='relu',
        input_shape=(X_train_norm.shape[1], 1)
    ))

    # Second Conv layer
    model.add(Conv1D(
        filters=hp.Int('conv2_filters', min_value=16, max_value=64, step=16),
        kernel_size=hp.Choice('conv2_kernel', values=[2, 3, 5]),
        activation='relu'
    ))

    model.add(Flatten())

    # Dense layers
    model.add(Dense(
        units=hp.Int('dense1_units', min_value=32, max_value=128, step=32),
        activation='relu'
    ))
    model.add(Dropout(hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1)))

    model.add(Dense(
        units=hp.Int('dense2_units', min_value=16, max_value=64, step=16),
        activation='relu'
    ))

    model.add(Dense(1, activation='sigmoid'))

    # Optimizer
    lr = hp.Choice('learning_rate', values=[1e-4, 5e-4, 1e-3, 5e-3])
    optimizer = Adam(learning_rate=lr)

    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

In [30]:
# ========================================
# STEP 4: HYPERPARAMETER TUNING
# ========================================
print("\n🔍 Starting hyperparameter tuning...")
print("  (This may take several minutes)")

tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=50,
    factor=3,
    directory='cnn_tuning',
    project_name='gallstone_cnn_corrected',
    overwrite=True  # Start fresh
)

# Early stopping to prevent overfitting
stop_early = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# CORRECT: Use validation set (NOT test set!)
tuner.search(
    X_train_cnn, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_val_cnn, y_val),  # ✓ CORRECT!
    callbacks=[stop_early],
    verbose=1
)

Trial 90 Complete [00h 00m 11s]
val_accuracy: 0.8333333134651184

Best val_accuracy So Far: 0.875
Total elapsed time: 00h 11m 19s


In [31]:
# ========================================
# STEP 5: GET BEST HYPERPARAMETERS
# ========================================
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print("\n" + "="*60)
print("🏆 BEST HYPERPARAMETERS FOUND")
print("="*60)
print(f"Conv1 Filters: {best_hps.get('conv1_filters')}")
print(f"Conv1 Kernel: {best_hps.get('conv1_kernel')}")
print(f"Conv2 Filters: {best_hps.get('conv2_filters')}")
print(f"Conv2 Kernel: {best_hps.get('conv2_kernel')}")
print(f"Dense1 Units: {best_hps.get('dense1_units')}")
print(f"Dense2 Units: {best_hps.get('dense2_units')}")
print(f"Dropout: {best_hps.get('dropout')}")
print(f"Learning Rate: {best_hps.get('learning_rate')}")



🏆 BEST HYPERPARAMETERS FOUND
Conv1 Filters: 64
Conv1 Kernel: 3
Conv2 Filters: 32
Conv2 Kernel: 2
Dense1 Units: 96
Dense2 Units: 16
Dropout: 0.30000000000000004
Learning Rate: 0.001


In [32]:
# ========================================
# STEP 6: TRAIN FINAL MODEL
# ========================================
print("\n🚀 Training final model with best hyperparameters...")

# Combine train + val for final training
X_trainval_norm = scaler_minmax.fit_transform(X_trainval)
X_trainval_cnn = X_trainval_norm.reshape(X_trainval_norm.shape[0], X_trainval_norm.shape[1], 1)

final_model = tuner.hypermodel.build(best_hps)
history = final_model.fit(
    X_trainval_cnn, y_trainval,
    epochs=50,
    batch_size=32,
    validation_split=0.2,  # 20% of trainval for monitoring
    callbacks=[stop_early],
    verbose=1
)


🚀 Training final model with best hyperparameters...
Epoch 1/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 0.4947 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6914
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.5697 - loss: 0.6849 - val_accuracy: 0.4792 - val_loss: 0.6920
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.6851 - loss: 0.6667 - val_accuracy: 0.4792 - val_loss: 0.6908
Epoch 4/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.6540 - loss: 0.6581 - val_accuracy: 0.5000 - val_loss: 0.6860
Epoch 5/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.6297 - loss: 0.6375 - val_accuracy: 0.5000 - val_loss: 0.6871
Epoch 6/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.6218 - loss: 0.6403 - val_accuracy: 0.5000 - val_loss: 0.6795
Epoch 7/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.6465 - loss: 0.6183 - val_accuracy: 0.5208 - val_loss: 0.6734
Epoch 8/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.6576 - lo

In [33]:
#========================================
# STEP 7: FINAL EVALUATION (FIRST TIME TOUCHING TEST SET!)
# ========================================
print("\n" + "="*60)
print("📈 FINAL TEST SET EVALUATION")
print("="*60)

test_loss, test_acc = final_model.evaluate(X_test_cnn, y_test, verbose=0)
y_pred_proba = final_model.predict(X_test_cnn, verbose=0).ravel()
y_pred = (y_pred_proba > 0.5).astype(int)

from sklearn.metrics import classification_report, confusion_matrix

print(f"\n✅ Test Accuracy: {test_acc*100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")

print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred, target_names=['Gallstone', 'No Gallstone']))

print("\n🔲 Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))



📈 FINAL TEST SET EVALUATION

✅ Test Accuracy: 85.00%
Test Loss: 0.3728

📊 Classification Report:
              precision    recall  f1-score   support

   Gallstone       0.91      0.78      0.84        40
No Gallstone       0.80      0.93      0.86        40

    accuracy                           0.85        80
   macro avg       0.86      0.85      0.85        80
weighted avg       0.86      0.85      0.85        80


🔲 Confusion Matrix:
[[31  9]
 [ 3 37]]


In [34]:

# ========================================
# STEP 8: SAVE MODEL
# ========================================
final_model.save('best_gallstone_cnn_model.h5')
print("\n💾 Model saved as 'best_gallstone_cnn_model.h5'")


💾 Model saved as 'best_gallstone_cnn_model.h5'


In [35]:
# ========================================
# VALIDATION CHECK
# ========================================
print("\n" + "="*60)
print("✓ VALIDATION CHECKLIST")
print("="*60)
print("✓ Test set was NOT used during hyperparameter tuning")
print("✓ Separate validation set was used for tuning")
print("✓ Final model trained on train+val data")
print("✓ Test set used ONLY for final evaluation")
print("✓ This is the CORRECT way to do hyperparameter tuning!")


✓ VALIDATION CHECKLIST
✓ Test set was NOT used during hyperparameter tuning
✓ Separate validation set was used for tuning
✓ Final model trained on train+val data
✓ Test set used ONLY for final evaluation
✓ This is the CORRECT way to do hyperparameter tuning!
